In [2]:
import pandas as pd
import numpy as np

from sklearn import svm
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

# Data

In [5]:
df = pd.read_csv("./data/clean_data.csv")
df = df.apply(pd.to_numeric,errors = "coerce")
print(df.shape)
df.head(5)

(7931, 9)


,date,TCMNOM_Y2,TCMNOM_Y10,DCOILBRENTEU,spindx,GOLDPMGBD228NLBM,exalus,exjpus,exukus
0,19870520,8.05,8.92,18.63,278.21,475.25,1.3820,139.78,0.5942
1,19870521,8.06,8.87,18.45,280.17,469.10,1.3864,140.45,0.5956
2,19870522,7.97,8.78,18.55,282.16,466.50,1.3883,140.60,0.5977
3,19870526,7.79,8.55,18.63,289.11,449.00,1.4029,143.30,0.6120
4,19870527,7.84,8.60,18.60,288.73,449.50,1.3988,143.52,0.6200


In [6]:
train = []
out = []
test = []
test_out = []
for i in np.random.randint(df.shape[0]-50, size=1000):
    if np.random.randint(100) % 5 == 0:
        test.append(np.array([df.iloc[c,1:].astype(float).values for c in range(i, i+30)]).T)
        test_out.append([df.iloc[i+35, 1:].astype(float).values])
    else:
        train.append(np.array([df.iloc[c,1:].astype(float).values for c in range(i, i+30)]).T)
        out.append([df.iloc[i+35, 1:].astype(float).values])

In [7]:
ts=np.random.uniform(-1,1,(50,3))
ts.shape

(50, 3)

In [8]:
ts.shape

(50, 3)

# Model

In [10]:
import ESN as res


states generated
Model Fitted


In [11]:
class ESN:
    
    def __init__(self,w,v,X='None',Activation=np.tanh,alpha=1,beta=0,PCA=False,n_components=100,UseLinear=True,readout=None):
        self.w=w
        self.v=v
        self.X=X
        self.Activation=Activation
        self.alpha=alpha
        self.beta=beta
        self.PCA=PCA
        self.n_components=n_components
        self.states=[]
        if UseLinear==False:
            self.readout=readout
        self.UseLinear=UseLinear
        self.n=v.shape[0]
        
    def GenNetState(self,TS,noise=True):
        if type(self.X)==str:
            self.X=np.matrix([0]*self.w.shape[0]).reshape((self.w.shape[0],1))
        states=[]
        for i in range(len(TS)):
            self.X=self.Activation(np.dot(self.w,self.X)+np.dot(self.v,TS[i]))
            states+=[self.X]
            self.states+=[self.X]
        return np.matrix(np.transpose(np.array(states),[2,0,1])[0])
    
    def fit(self,inTS,out,score=False):
        '''input a sequence X and a target sequence y to fit weights'''
        states = self.GenNetState(inTS)
        if self.UseLinear==True:
            if self.alpha==0 and self.beta==0:
                self.readout=linear_model.LinearRegression()
                
            elif self.alpha==0:
                self.readout = linear_model.Lasso(alpha=self.beta)
                
            elif self.beta==0:
                self.readout = linear_model.Ridge(alpha=self.alpha)
                
            else:
                self.readout = linear_model.ElasticNet(alpha=self.alpha,l1_ratio=self.beta/self.alpha)
            
        if self.PCA==True:
            self.meansub=states.mean(0).reshape(1,self.n)
            states=states-self.meansub
            U,sigma,W=np.linalg.svd(states)
            self.Wtrunk=W.T[:self.n_components].T
            states=states*self.Wtrunk
            
        self.readout.fit(states,out)
        
        if self.UseLinear==True:
            self.Weights=np.matrix(self.readout.coef_).T
            self.intercept=self.readout.intercept_

        if score==True:
            return [self.Weights,self.readout.score(states,out)]


    def pca(self,X):
        if self.PCA==False:
            return X
        else:
            return (X-self.meansub)*self.Wtrunk
    
    def kStepFit(self,TS,k,score=False):
        if score==False:
            self.fit(TS[:-k],TS[k:],score=score)
        else:
            return self.fit(TS[:-k],TS[k:],score=score)[1]
        
    def predict(self,inTS):
        states=self.GenNetState(inTS)
        states = self.pca(states)
        
        return self.readout.predict(states)
    
    def FreeRun(self,length):
        states=[]
        for i in range(length):
            pred=self.readout.predict(self.pca(self.X.T))[0]
            self.X=self.Activation(np.dot(self.w,self.X)+np.dot(self.v,pred))
            states+=[self.X]
        stateMat=np.transpose(np.array(states),[2,0,1])[0]
        preds=self.readout.predict(self.pca(stateMat))
        return preds


20 Node network, with 3 dimensional input

In [12]:
n_nodes=20
dim_input=3
W = res.RandomMatrix(n_nodes, 0, 0.9)
v = np.matrix(np.random.uniform(-1,1,size=n_nodes*dim_input)).reshape((n_nodes,dim_input))
net = ESN(W,v,Activation=np.tanh)

In order to fit, we want to predict 1 step ahead, so we truncate the last entry of the original time series, and shift it by 1 for the target

a,b,c,d,e,f

becomes

a,b,c,d,e

predicting

b,c,d,e,f

In [13]:
net.fit(ts[:-1],ts[1:])

In [14]:
net.FreeRun(10)

array([[-0.03663138, -0.22347683, -0.00626524],
       [-0.1396812 ,  0.09007733, -0.16398958],
       [-0.1893055 ,  0.40425434, -0.12397461],
       [-0.3746174 ,  0.51884209, -0.05019251],
       [-0.45044801,  0.25502108, -0.13255163],
       [-0.386311  ,  0.09062258, -0.16373426],
       [-0.23102869, -0.08935983, -0.11169046],
       [-0.11962778, -0.33259385, -0.01518283],
       [ 0.01441911, -0.51971515,  0.01543344],
       [ 0.15256603, -0.56194188,  0.06063159]])